# Big Mart

# Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import warnings
warnings.filterwarnings("ignore")

# Checking null values

In [2]:
data=pd.read_csv(r"C:\Users\Aysha Riya\OneDrive\Documents\bigmart.csv")

In [3]:
data

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
...,...,...,...,...,...,...,...,...,...,...,...,...
8518,FDF22,6.865,Low Fat,0.056783,Snack Foods,214.5218,OUT013,1987,High,Tier 3,Supermarket Type1,2778.3834
8519,FDS36,8.380,Regular,0.046982,Baking Goods,108.1570,OUT045,2002,NaN,Tier 2,Supermarket Type1,549.2850
8520,NCJ29,10.600,Low Fat,0.035186,Health and Hygiene,85.1224,OUT035,2004,Small,Tier 2,Supermarket Type1,1193.1136
8521,FDN46,7.210,Regular,0.145221,Snack Foods,103.1332,OUT018,2009,Medium,Tier 3,Supermarket Type2,1845.5976


In [4]:
data.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [6]:
data.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
8518    False
8519    False
8520    False
8521    False
8522    False
Length: 8523, dtype: bool

In [7]:
data['Item_Weight'].fillna(data['Item_Weight'].mean(), inplace=True)

In [8]:
data['Outlet_Size'].fillna(data['Outlet_Size'].mode()[0], inplace=True)

In [9]:
data.isna().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

In [10]:
data.to_excel("cleaned_bigmart.xlsx")

# Encoding

In [11]:
d = pd.read_excel("cleaned_bigmart.xlsx")

In [12]:
from sklearn.preprocessing import LabelEncoder

In [13]:
# Binary Encoding
l=LabelEncoder()
d['Item_Fat_Content']=l.fit_transform(d['Item_Fat_Content'])

pickle.dump(l, open("l_item_fat.pkl", "wb"))

# Ordinal Encoding
d['Outlet_Size'] = d['Outlet_Size'].map({'Small': 0, 'Medium': 1, 'High': 2})
d['Outlet_Location_Type'] = d['Outlet_Location_Type'].map({'Tier 1': 0, 'Tier 2': 1, 'Tier 3': 2})

# One-Hot Encoding
d = pd.get_dummies(d, columns=['Item_Type', 'Outlet_Type'], drop_first=True)

x_columns = d.drop('Item_Outlet_Sales', axis=1).columns  # all feature columns
pickle.dump(x_columns, open("x_columns.pkl", "wb"))


# Train_Test split

In [14]:
x = d.drop(['Item_Outlet_Sales','Item_Identifier','Item_MRP','Outlet_Establishment_Year','Outlet_Identifier'], axis=1)
y = d['Item_Outlet_Sales']

In [15]:
from sklearn.model_selection import train_test_split
xtrain,xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=42)

# Model Building

In [16]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# linear regression

In [17]:
le=LinearRegression()
le.fit(xtrain,ytrain)
ypred=le.predict(xtest)

In [18]:
print("MAE:",mean_absolute_error(ytest,ypred))
print("MSE:",np.sqrt(mean_squared_error(ytest,ypred)))
print("R2:",r2_score(ytest,ypred))

MAE: 1066.7873415229847
MSE: 1413.7049072589662
R2: 0.26468595465591216


# decision tree regressor

In [19]:
dt=DecisionTreeRegressor()
dt.fit(xtrain,ytrain)
ypred=dt.predict(xtest)

In [20]:
print("MAE:",mean_absolute_error(ytest,ypred))
print("MSE:",np.sqrt(mean_squared_error(ytest,ypred)))
print("R2:",r2_score(ytest,ypred))

MAE: 1349.5418456304988
MSE: 1911.3852033511914
R2: -0.3441624837824464


# random forest regressor

In [21]:
rf=RandomForestRegressor()
rf.fit(xtrain,ytrain)
ypred=rf.predict(xtest)

In [22]:
print("MAE:",mean_absolute_error(ytest,ypred))
print("MSE:",np.sqrt(mean_squared_error(ytest,ypred)))
print("R2:",r2_score(ytest,ypred))

MAE: 1016.5605516457479
MSE: 1378.8096723818555
R2: 0.3005382450660199


- Random Forest is the most reliable model overall
- Among the models tested, the Random Forest Regressor performs the best overall, showing higher R² and lower error metrics.
- Therefore, Random Forest is the most accurate and reliable model for predicting Item_Outlet_Sales.

# saving model

In [23]:
import joblib
joblib.dump(rf, "big_mart.pkl")

['big_mart.pkl']